## Installing Sentence Transformer and other models/frameworks

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install sentence_transformers
!pip install fasttext
!pip install transformers 
# Kindly add all your installations and versions if any in this cell.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing necessary libraries. 
In the final version all imports should be stricly enlisted here.

In [3]:
###
import pandas as pd
import numpy as np
# import spacy
from scipy import stats
from sklearn import linear_model

# from sentence_transformers import SentenceTransformer, losses, models, util
# from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
# from sentence_transformers.readers import InputExample

import torch 
from torch.utils.data import DataLoader

####

import os
import fasttext.util
import nltk
nltk.download('punkt')
import csv

from scipy.stats import spearmanr
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler  

from torch import nn
from torch.utils.data import Dataset
from transformers import BertModel
from transformers import BertTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
###
os.chdir('/content/drive/MyDrive/')
# os.mkdir('NLP_Ass3')
os.chdir('NLP_Ass3')
root = '/content/drive/MyDrive/NLP_Ass3'

## Load dataset: 7 marks
1 Download and unzip the dataset from this link http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz  **1 mark**

2 Complete the code in `read_sts_csv()`. **4.5 marks**

3 Create 3 dataframes one each for train, test and val and print their final shapes. **1.5 marks**

In [ ]:
# os.chdir(root)
# !wget http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
# !ls

In [13]:
os.chdir(root)
# !tar -xvf /content/drive/MyDrive/NLP_Ass3/Stsbenchmark.tar.gz
#make a small edit in the .csv of 'sts-test.csv' - SOLVED
#train.csv line number 2100 - SOLVED

In [5]:
###
os.chdir(root)
def read_sts_csv(dataset_type="train", columns=['source', 'type', 'year', 'id', 'score', 'sent_a', 'sent_b']):
    path = root+'/stsbenchmark/'+"sts-"+dataset_type+".csv"
    return pd.read_csv(path, header=None, usecols=columns, names=columns, sep='\t', quoting=csv.QUOTE_NONE)

df_train = read_sts_csv() # create the train, dev and test dataframes
df_dev = read_sts_csv('dev')
df_test = read_sts_csv('test')

print("Train data-frame shape",df_train.to_numpy().shape)
print("Dev data-frame shape",df_dev.to_numpy().shape)
print("Test data-frame shape",df_test.to_numpy().shape)

Train data-frame shape (5749, 7)
Dev data-frame shape (1500, 7)
Test data-frame shape (1379, 7)


## Hyperparameters: 5 Marks
Update this cell with you choosen parameters except, NUM_EPOCHS

In [ ]:
os.chdir(root)

CONTEXTUAL_MODEL_TYPE = 'bert-base-uncased'
# HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL = <HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL> # USE THE HUGGAING FACE VERSION OF SENTENCE_TRANSFORMER_TYPE
# INPUT_PATH = <INPUT_FOLDER_PATH>
BATCH_SIZE = 8
# OUT_DIM_DENSE = <OUT_DIM_DENSE>
NUM_EPOCHS = 2 ## THIS IS FIXED DO NOT CHANGE

# You are free to add your own hyperparameters as well.

In [ ]:
os.chdir(root)

# fasttext.util.download_model('en', if_exists='ignore')
NON_CONTEXTUAL_MODEL_TYPE = fasttext.load_model('cc.en.300.bin') #fast-text as non-contextual model

## CONFIGURATION 1: Non-contextual Embeddings + ML Regression: 8 marks
1 Load the non-contextual embedding model in variable `non_cont_model1`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model1()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model1`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model1`. **1.5 mark**



In [ ]:
def get_feature_model1(data_frame):
    """
    Input a data frame and return the embedding vectors for the each sentence column using non_cont_model1,
    Return 2 matrices each of shape (#_samples, #size_of_word_emb).
    """
    dataset_np_array = data_frame.to_numpy()

    embeddings_1 = []
    for i in range(dataset_np_array.shape[0]):
        word_tokens_1 = nltk.tokenize.word_tokenize(dataset_np_array[i][5])
        mean_embedding_1 = np.zeros((300,)) #word embeddings in 300 dimensions
        for token in word_tokens_1:
            mean_embedding_1 += non_cont_model1.get_word_vector(token)
        mean_embedding_1 /= len(word_tokens_1)
        embeddings_1.append(mean_embedding_1)

    embeddings_2 = []
    for i in range(dataset_np_array.shape[0]):
        word_tokens_2 = nltk.tokenize.word_tokenize(dataset_np_array[i][6])
        mean_embedding_2 = np.zeros((300,)) #word embeddings in 300 dimensions
        for token in word_tokens_2:
            mean_embedding_2 += non_cont_model1.get_word_vector(token)
        mean_embedding_2 /= len(word_tokens_2)
        embeddings_2.append(mean_embedding_2)

    return np.array(embeddings_1), np.array(embeddings_2)

non_cont_model1 = NON_CONTEXTUAL_MODEL_TYPE

feature_1_train, feature_2_train = get_feature_model1(df_train)
print("Train shapes:", feature_1_train.shape, feature_2_train.shape)
feature_1_dev, feature_2_dev = get_feature_model1(df_dev)
print("Dev shapes:", feature_1_dev.shape, feature_2_dev.shape)
feature_1_test, feature_2_test = get_feature_model1(df_test)
print("Test shapes:", feature_1_test.shape, feature_2_test.shape)

Train shapes: (5749, 300) (5749, 300)
Dev shapes: (1500, 300) (1500, 300)
Test shapes: (1379, 300) (1379, 300)


In [ ]:
# X_train, Y_train = np.concatenate((feature_1_train,feature_2_train), axis=1), df_train.to_numpy()[:,4]
# X_dev, Y_dev = np.concatenate((feature_1_dev,feature_2_dev), axis=1), df_dev.to_numpy()[:,4]
# X_test, Y_test = np.concatenate((feature_1_test,feature_2_test), axis=1), df_test.to_numpy()[:,4]

# MAX_DEGREE = 1
# NUM_COMPONENTS = 600

# polyreg = make_pipeline(PolynomialFeatures(MAX_DEGREE, include_bias=True), LinearRegression())
# pca = PCA(n_components=NUM_COMPONENTS)
# X_train_pca = pca.fit_transform(X_train)
# polyreg.fit(X_train_pca, Y_train)

# X_dev_and_test_pca = pca.fit_transform(np.concatenate((X_dev, X_test)))
# Y_predicted_dev_and_test = polyreg.predict(X_dev_and_test_pca)

# print("Spearman correlation:", stats.spearmanr(Y_predicted_dev_and_test, np.concatenate((Y_dev, Y_test))))
# print(Y_predicted_dev_and_test.round(2))

In [ ]:
X_train, Y_train = np.concatenate((feature_1_train,feature_2_train), axis=1), df_train.to_numpy()[:,4]
X_dev, Y_dev = np.concatenate((feature_1_dev,feature_2_dev), axis=1), df_dev.to_numpy()[:,4]
X_test, Y_test = np.concatenate((feature_1_test,feature_2_test), axis=1), df_test.to_numpy()[:,4]

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_dev = scaler.transform(X_dev)
X_test = scaler.transform(X_test)

regr = MLPRegressor(random_state=42, batch_size=BATCH_SIZE, solver='adam', max_iter=NUM_EPOCHS).fit(X_train, Y_train)

Y_predicted_dev = regr.predict(X_dev)
Y_predicted_test = regr.predict(X_test)


print("Spearman correlation dev:", stats.spearmanr(Y_predicted_dev, Y_dev))
print("Spearman correlation test:", stats.spearmanr(Y_predicted_test, Y_test))


Spearman correlation dev: SpearmanrResult(correlation=0.25068476886281293, pvalue=6.284517507581251e-23)
Spearman correlation test: SpearmanrResult(correlation=0.25313640089143147, pvalue=1.3275369550669051e-21)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


## CONFIGURATION 2: Contextual Embeddings + ML Regression: 7 marks
1 Load the contextual embedding model in variable `non_cont_model2`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model2()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model2`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model2`. **1.5 mark**

Useful references: https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [6]:
###
class CustomDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.scores = dataset[:,4]
        self.sent_1 = []
        self.sent_2 = []
        for i in range(dataset.shape[0]):
            self.sent_1.append(tokenizer(dataset[i,5], max_length=512, padding='max_length', truncation=True, return_tensors="pt"))
            self.sent_2.append(tokenizer(dataset[i,6], max_length=512, padding='max_length', truncation=True, return_tensors="pt"))
        # self.sent_1 = np.array(self.sent_1)
        # self.sent_2 = np.array(self.sent_2)

    def __len__(self):
        return self.scores.shape[0]

    def __getitem__(self, idx):
        return self.scores[idx], self.sent_1[idx], self.sent_2[idx]

    def getScores(self):
        return self.scores

In [ ]:
class BertEmbeddingGenerator(nn.Module):
    def __init__(self):
        super(BertEmbeddingGenerator, self).__init__()
        self.bert = BertModel.from_pretrained(CONTEXTUAL_MODEL_TYPE)

    def forward(self, input_ids, bert_mask):
        return self.bert(input_ids=input_ids, attention_mask=bert_mask, return_dict=False)[1]

In [ ]:
def getEmbeddings(model, data, device):
    model.eval()
    if device=='cuda' and torch.cuda.is_available():
        model.cuda()

    dataloader = DataLoader(data, batch_size=BATCH_SIZE)

    sents_1 = []
    sents_2 = []

    with torch.no_grad():
        for batch, (_, sent_1, sent_2) in enumerate(dataloader):
            output_1 = model(sent_1["input_ids"].squeeze(1).to(device), sent_1["attention_mask"].to(device))
            output_2 = model(sent_2["input_ids"].squeeze(1).to(device), sent_2["attention_mask"].to(device))
            # print(output_1.shape)
            for i in range(len(output_1)):
                sents_1.append(output_1[i].cpu())
                sents_2.append(output_2[i].cpu())
    
    sents_1 = np.array(sents_1)
    sents_2 = np.array(sents_2)
    print(sents_1.shape, sents_2.shape)

    return sents_1, sents_2

In [ ]:
def get_feature_model2(data_frame):
    """
    Input a data frame and return the embedding vectors for the each sentence column using model2,
    Return 2 matrices each of shape (#_samples, #size_of_word_emb).
    """
    BATCH_SIZE = 8
    device = 'cuda'
    model = BertEmbeddingGenerator()
    # print(non_cont_model2)
    dataset = CustomDataset(data_frame.to_numpy(), BertTokenizer.from_pretrained(non_cont_model2))
    return getEmbeddings(model, dataset, device)



non_cont_model2 = CONTEXTUAL_MODEL_TYPE

feature_1_train, feature_2_train = get_feature_model2(df_train)
print("Train shapes:", feature_1_train.shape, feature_2_train.shape)
feature_1_dev, feature_2_dev = get_feature_model2(df_dev)
print("Dev shapes:", feature_1_dev.shape, feature_2_dev.shape)
feature_1_test, feature_2_test = get_feature_model2(df_test)
print("Test shapes:", feature_1_test.shape, feature_2_test.shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the 

(5749,) (5749,)
Train shapes: (5749,) (5749,)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1500,) (1500,)
Dev shapes: (1500,) (1500,)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1379,) (1379,)
Test shapes: (1379,) (1379,)


In [ ]:
Y_train = df_train.to_numpy()[:,4]
X_train = [np.concatenate((feature_1_train[i], feature_2_train[i]), axis=0) for i in range(feature_1_train.shape[0])]
X_train = np.array(X_train)
print(X_train.shape, Y_train.shape)
Y_dev =  df_dev.to_numpy()[:,4]
X_dev = [np.concatenate((feature_1_dev[i], feature_2_dev[i]), axis=0) for i in range(feature_1_dev.shape[0])]
X_dev = np.array(X_dev)
print(X_dev.shape, Y_dev.shape)
Y_test = df_test.to_numpy()[:,4]
X_test = [np.concatenate((feature_1_test[i], feature_2_test[i]), axis=0) for i in range(feature_1_test.shape[0])]
X_test = np.array(X_test)
print(X_test.shape, Y_test.shape)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_dev = scaler.transform(X_dev)
X_test = scaler.transform(X_test)

BATCH_SIZE = 24

regr = MLPRegressor(random_state=42, batch_size=BATCH_SIZE, solver='adam', max_iter=NUM_EPOCHS).fit(X_train, Y_train)
Y_predicted_dev = regr.predict(X_dev)
Y_predicted_test = regr.predict(X_test)


print("Spearman correlation dev:", stats.spearmanr(Y_predicted_dev, Y_dev))
print("Spearman correlation test:", stats.spearmanr(Y_predicted_test, Y_test))


(5749, 1536) (5749,)
(1500, 1536) (1500,)
(1379, 1536) (1379,)
Spearman correlation dev: SpearmanrResult(correlation=0.21481719768780774, pvalue=4.055571443668732e-17)
Spearman correlation test: SpearmanrResult(correlation=0.2168585462890702, pvalue=3.8623145265734496e-16)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


## CONFIGURATION 3: Fine-Tune a Contextual Embeddings Model: 18 marks
1 Prepare data samples to be for the DL model to consume. Add the code in the `form_data()`. **4 marks**

3 Create the data loader, one each for train/dev/test data_input sample set obtained from `form_input_example()`. **1.5 marks**

4 Initiate `model3` consisting of **atleast** the following 3 components - `base_LM`, a `pooling_layer` and a `dense_layer`. Use appropriate activation function in dense. **Atleast** one layer of `base_LM` should be set to trainable. **5 marks**

6 Initiate the `loss`. **0.5 marks**

7 Fit the `model3`. Use `NUM_EPOCHS = 2`. **MAX_NUM_EPOCHS allowed will be 3**. **2 marks** 

8 Complete the `get_model_predicts()` to obtain predicted scores for input sentence pairs. **3.5 marks** 

9 Print the correlation scores on the dev and test set predictions. **1.5 mark**

Useful References: https://huggingface.co/blog/how-to-train-sentence-transformers 

In [26]:
###
class BertSemantic(nn.Module):
    def __init__(self): ## BERT POOL DENSE
        super(BertSemantic, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.avg_pool = nn.AvgPool1d(10, stride=2)
        # self.linear_layer_1 = nn.LazyLinear(256)
        self.linear_layer = nn.LazyLinear(256)
        # self.linear_layer_2 = nn.Linear(32, 10)
        # self.linear_layer_3 = nn.Linear(10, 1)
        self.relu = nn.ReLU()
        self.tan_h = nn.Tanh()
        self.cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)

    def forward(self, input_ids_1, bert_mask_1, input_ids_2, bert_mask_2):
        _, pooled_output_1 = self.bert(input_ids=input_ids_1, attention_mask=bert_mask_1, return_dict=False)
        # print(pooled_output_1.shape)
        pooled_cls_1 = self.avg_pool(pooled_output_1.reshape(1, pooled_output_1.shape[0], -1))
        _, pooled_output_2 = self.bert(input_ids=input_ids_2, attention_mask=bert_mask_2, return_dict=False)
        # print(pooled_output_1.shape)
        pooled_cls_2 = self.avg_pool(pooled_output_2.reshape(1, pooled_output_2.shape[0], -1))

        pooled_cls_1 = pooled_cls_1.squeeze(dim=0)
        lin_cls_1 = self.tan_h(self.linear_layer(pooled_cls_1))

        pooled_cls_2 = pooled_cls_2.squeeze(dim=0)
        lin_cls_2 = self.tan_h(self.linear_layer(pooled_cls_2))

        return self.cos_sim(lin_cls_1, lin_cls_2)
        # return self.relu(self.linear_layer(pooled_cls))


In [27]:
###

def form_data(data_frame, data_type):
    """
    Input a data frame and return the dataloder.
    """
    # print(non_cont_model2)
    dataset = CustomDataset(data_frame.to_numpy(), BertTokenizer.from_pretrained("bert-base-uncased"))
    if data_type == "train":
        return DataLoader(dataset, batch_size=8, shuffle=True)
    return DataLoader(dataset, batch_size=8)


def get_model_predicts(data_type, trained_model):
    """
    Input the dataset list and return a list of cosine similarity scores. Use the fitted final_trainable_model for obtaining encodings.
    """
    BATCH_SIZE = 8
    device = 'cuda'
    if(data_type == "dev"):
        return evalModel(trained_model, form_data(df_dev, "dev"), device)
    elif(data_type == "test"):
        return evalModel(trained_model, form_data(df_test, "test"), device)
    elif(data_type == "train"):
        return evalModel(trained_model, form_data(df_train, "train"), device)




def evalModel(model, dataloader, device):
    model.eval()
    if(device == "cuda"):
        model.cuda()
    
    pred_scores = []
    with torch.no_grad():
        for batch, (scores, sent_1, sent_2) in enumerate(dataloader):
            output = model(sent_1["input_ids"].squeeze(1).to(device), sent_1["attention_mask"].to(device), sent_2["input_ids"].squeeze(1).to(device), sent_2["attention_mask"].to(device))

            for i in range(len(output)):
                pred_scores.append(output[i].cpu())
    model.train()
    return  np.array(pred_scores)

def trainModel(model, train_dataloader, learning_rate_bert, learning_rate, epochs, device):
    model.train()
    loss_function = nn.MSELoss()
    optimizer_bert = torch.optim.Adam(model.parameters(), lr=learning_rate_bert)
    optimizer_lin_1 = torch.optim.Adam(model3.linear_layer.parameters(), lr=learning_rate)
    # optimizer_lin_2 = torch.optim.Adam(model3.linear_layer_2.parameters(), lr=learning_rate)
    # optimizer_lin_3 = torch.optim.Adam(model3.linear_layer_3.parameters(), lr=learning_rate)

    if(device == "cuda"):
        model.cuda()
        loss_function.cuda()

    for epoch in range(epochs):
        for batch, (scores, sent_1, sent_2) in enumerate(train_dataloader):
            output = model(sent_1["input_ids"].squeeze(1).to(device), sent_1["attention_mask"].to(device), sent_2["input_ids"].squeeze(1).to(device), sent_2["attention_mask"].to(device))
            # print("output shape: ", output.shape)
            # print(output.dtype)
            loss = loss_function(output.squeeze(), scores.to(device).to(output.dtype).squeeze()/5)
            # print(scores.to(device).dtype)
            if batch%50 == 0:
                print(f"epoch: {epoch+1}, loss of current batch: {loss}")
            # update 
            optimizer_bert.zero_grad()
            optimizer_lin_1.zero_grad()
            # optimizer_lin_1.zero_grad()
            # optimizer_lin_2.zero_grad()
            # optimizer_lin_3.zero_grad()
            loss.backward()
            optimizer_bert.step()
            optimizer_lin_1.step()
            # optimizer_lin_2.step()
            # optimizer_lin_3.step()

        print(f"EPOCH {epoch + 1} done!")


        pred_dev = get_model_predicts("dev", model3)
        pred_dev = [pred for pred in pred_dev]
        Y_dev =  df_dev.to_numpy()[:,4]
        print("Spearman correlation dev:", stats.spearmanr(pred_dev, Y_dev))



# # dataloader_<dataset_type> = form_data(data_frame)
# dataloader_train = form_data(df_train, "train")
# # base_model = 
# model3 = BertSemantic()
# # layer_ppoling = 
# layer_dense = 
# model3 = 
# loss =



In [28]:
dataloader_train = form_data(df_train, "train")
# base_model = 
model3 = BertSemantic()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the 

In [20]:
# 205
print(len([ para  for para in model3.parameters()]))
print(len([ para  for para in model3.linear_layer.parameters()]))
# print(len([ para  for para in model3.linear_layer_1.parameters()]))
# print(len([ para  for para in model3.linear_layer_2.parameters()]))
# print(len([ para  for para in model3.linear_layer_3.parameters()]))
print(len([ para  for para in model3.bert.parameters()]))

201
2
199


In [29]:
# Fit the model3.
trainModel(model3, dataloader_train, 0.00005, 0.001, 1, "cuda")

epoch: 1, loss of current batch: 0.08521401137113571
epoch: 1, loss of current batch: 0.14950960874557495
epoch: 1, loss of current batch: 0.33640211820602417
epoch: 1, loss of current batch: 0.26294732093811035
epoch: 1, loss of current batch: 0.09796900302171707


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-5ea2d2a3258a>", line 2, in <module>
    trainModel(model3, dataloader_train, 0.00005, 0.001, 1, "cuda")
  File "<ipython-input-27-60ac1b6463d6>", line 72, in trainModel
    loss.backward()
  File "/usr/local/lib/python3.7/dist-packages/torch/_tensor.py", line 396, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py", line 175, in backward
    allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2040

KeyboardInterrupt: ignored

In [23]:
pred_dev = get_model_predicts("dev", model3)
pred_test = get_model_predicts("test", model3)
# pred_train = get_model_predicts("train", model3)
Y_dev =  df_dev.to_numpy()[:,4]
# Y_train = df_train.to_numpy()[:,4]
Y_test = df_test.to_numpy()[:,4]
pred_dev = [pred for pred in pred_dev]
pred_test = [pred for pred in pred_test]

In [25]:
# Print spearman correlation on the predicted output of the dev and test sets.
print(pred_dev[20:30])
print(Y_dev.shape)
print("Spearman correlation dev:", stats.spearmanr(pred_dev, Y_dev, nan_policy='omit'))
print("Spearman correlation test:", stats.spearmanr(pred_test, Y_test, nan_policy='omit'))

[0.943133, 0.7712677, 0.67079604, 0.504155, 0.30420354, 0.80869627, 0.8496884, 0.5772686, 0.36969975, 0.12360539]
(1500,)
Spearman correlation dev: SpearmanrResult(correlation=0.6816948352135648, pvalue=1.5658668824475524e-205)
Spearman correlation test: SpearmanrResult(correlation=0.6267292332147769, pvalue=2.3115458342051015e-151)


In [ ]:
print(pred_dev)
# print(pred_dev.shape)
# print(pred_dev.numpy())
# print(Y_dev.shape)

[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=f

In [ ]:
!nvidia-smi

Mon Nov 28 09:37:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------